# Alignment of MusicBrainz and Wikidata Record Labels

In [ ]:
%run -i ../startup.py
import numpy as np
ENTITY_TYPE = 'label'

Wikidata entities:

https://www.wikidata.org/wiki/Q18127 record label

Wikidata properties:

https://www.wikidata.org/wiki/Property:P214 VIAF

https://www.wikidata.org/wiki/Property:P268 BNF

https://www.wikidata.org/wiki/Property:P244 LoC

https://www.wikidata.org/wiki/Property:P1955 discogs ID

https://www.wikidata.org/wiki/Property:P966 MusicBrainz label ID

examples

https://www.wikidata.org/wiki/Q885833


## Record labels from Wikidata

In [ ]:
links_type_from_wd = sparql("""
SELECT distinct (count(?label) as ?cnt) ?ins ?insLabel
WHERE {
  ?label wdt:P31 ?ins;
    wdt:P966 ?mbid.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
group by ?ins ?insLabel
order by DESC(?cnt)
""")
links_type_from_wd[links_type_from_wd.cnt.astype(int) > 10]

In [ ]:
# linked to MB label
links_from_wd = sparql("""
SELECT (?label AS ?wd) ?mbid ?labelLabel ?lc ?viaf ?discogs
WHERE {
  ?label wdt:P966 ?mbid .
  OPTIONAL { ?label wdt:P7320 ?lc . }
#  OPTIONAL { ?label wdt:P214 ?viaf . }
#  OPTIONAL { ?label wdt:P1955 ?discogs . }
#  OPTIONAL { ?label wdt:P268 ?bnf . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
ORDER BY ASC(?labelLabel)
""")
links_from_wd.rename(columns={'labelLabel': 'name'}, inplace=True)
links_from_wd.lc = links_from_wd.lc.apply(
    lambda lc: lc if isinstance(lc, str) else '')
#links_from_wd.discogs = links_from_wd.discogs.apply(
#    lambda discogs: discogs if isinstance(discogs, str) else '')

print('Count:', len(links_from_wd))
display_df(links_from_wd.head())

## Labels from MusicBrainz with wikidata links

In [ ]:
links_from_mb = sql("""
SELECT
    url.url AS wd,
    label.gid AS mbid,
    label.name,
    label.label_code AS lc
FROM 
    label
    JOIN l_label_url AS llu ON llu.entity0 = label.id
    JOIN url                ON llu.entity1 = url.id
WHERE
    url.url LIKE '%%wikidata.org%%'
ORDER BY
    label.name
;
""")
links_from_mb.wd = links_from_mb.wd.apply(lambda s: s.split('/')[-1])
links_from_mb.mbid = links_from_mb.mbid.apply(str)
links_from_mb.lc = links_from_mb.lc.apply(
    lambda lc: '' if np.isnan(lc) else str(int(lc)))

print('Count:', len(links_from_mb))
display_df(links_from_mb.head())

## Duplicate links

In [ ]:
duplicate_wd = links_from_mb[[
    'wd', 'mbid', 'name']].groupby('wd').filter(
    lambda row: len(row.mbid) > 1).sort_values('wd')

print('Count:', len(duplicate_wd))
display_df(duplicate_wd.head())

In [ ]:
duplicate_mb = links_from_mb[['wd', 'mbid', 'name']].groupby('mbid').filter(
    lambda row: len(row.mbid) > 1).sort_values('mbid')

print('Count:', len(duplicate_mb))
display_df(duplicate_mb.head())

## Data alignment

In [ ]:
merge = pd.merge(links_from_wd, links_from_mb, 
                 on=['wd', 'mbid'], suffixes=('_wd', '_mb'),
                 how='outer', indicator=True)
display_df(merge.head())

In [ ]:
# link in mb but missing in wd
links_to_add_to_wd = merge.loc[lambda x : x['_merge']=='right_only'][[
    'name_mb', 'mbid', 'wd']]

print('Count:', len(links_to_add_to_wd))
display_df(links_to_add_to_wd.head())

In [ ]:
# link in wd but missing in mb
links_to_add_to_mb = merge.loc[lambda x : x['_merge']=='left_only'][[
    'name_wd', 'wd', 'mbid']]
links_to_add_to_mb['edit_link'] = links_to_add_to_mb.apply(
    mb_label_edit_wd_link, axis=1)

print('Count:', len(links_to_add_to_mb))
display_df(links_to_add_to_mb.head())

Example:
https://musicbrainz.org/label/85bb6180-ac99-46b5-a3ec-92967e88f842 is not linked to https://www.wikidata.org/wiki/Q56809543

https://musicbrainz.org/label/86e1fce2-a61e-4d08-9d4f-b91d602f995b is linked to https://www.wikidata.org/wiki/Q24950167 that was removed in https://www.wikidata.org/w/index.php?title=Special:Log&logid=661147435

https://musicbrainz.org/label/69de915d-e7b5-4739-bd21-2de1099a0610 is linked to https://www.wikidata.org/wiki/Q4146440 and not the opposite

### Add missing Label Codes

In [ ]:
lc_to_add_to_mb = merge.loc[
    (merge._merge=='both') & (merge.lc_mb == '') & (merge.lc_wd != '')
][['mbid', 'wd', 'lc_wd']].sort_values(by='lc_wd')
lc_to_add_to_mb['edit_link'] = lc_to_add_to_mb.apply(
    mb_label_edit_lc_link, axis=1)

print('Count:', len(lc_to_add_to_mb))
display_df(lc_to_add_to_mb)

In [ ]:
# linked to MB label
links_from_wd_with_bnf = sparql("""
SELECT (?label AS ?wd) ?mbid ?labelLabel ?bnf
WHERE {
  ?label wdt:P966 ?mbid .
  ?label wdt:P268 ?bnf .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
ORDER BY ASC(?labelLabel)
""")
links_from_wd_with_bnf.rename(columns={'labelLabel': 'name'}, inplace=True)

print('Count:', len(links_from_wd_with_bnf))
display_df(links_from_wd_with_bnf.head())

In [ ]:
links_from_mb_with_bnf = sql("""
SELECT
    url.url AS wd,
    label.gid AS mbid,
    label.name
FROM label
JOIN l_label_url AS llu             ON llu.entity0 = label.id
JOIN url                            ON llu.entity1 = url.id
WHERE
    url.url LIKE '%%bnf.fr%%'
ORDER BY label.name
;
""")
links_from_mb_with_bnf.wd = links_from_mb.wd.apply(lambda s: s.split('/')[-1])
links_from_mb_with_bnf.mbid = links_from_mb.mbid.apply(str)

print('Count:', len(links_from_mb_with_bnf))
display_df(links_from_mb_with_bnf.head())

report
x missing WD link
x missing LC / diverging LC
missing discogs /div 
missing viaf / div
missing bnf / div

common LC but no direct MB/WD link
common discogs viaf bnf

wikipedia link and no wikidata link

## Report

In [ ]:
import jinja2

template = jinja2.Template("""
<!doctype html>

<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Alignment of MusicBrainz and Wikidata Record Labels</title>
    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
  </head>

  <body style="margin: 20px;">
    <h1>Alignment of MusicBrainz and Wikidata Record Labels</h1>

    <p>Latest MB database update: {{ MB_DATABASE_VERSION }}</p>
    <p>Latest update: {{ date.today().isoformat() }}</p>

    <ol>
      <li>
        <a href="#lc">Add missing Label Codes</a>
        ({{ lc_to_add_to_mb.shape[0] }} rows)
      </li>
      <li>
        <a href="#wddup">MusicBrainz entities sharing a Wikidata link</a>
        ({{ duplicate_wd.shape[0] }} rows)
      </li>
      <li>
        <a href="#mbdup">Wikidata entities sharing a MusicBrainz link</a>
        ({{ duplicate_mb.shape[0] }} rows)
      </li>
      <li>
        <a href="#wd2mb">Add missing Wikidata links to MusicBrainz</a>
        ({{ links_to_add_to_mb.shape[0] }} rows)
      </li>
      <li>
        <a href="#mb2wd">Add missing MusicBrainz links to Wikidata</a>
        ({{ links_to_add_to_wd.shape[0] }} rows)
      </li>
    </ol>
    
    <h2 id="lc">Add missing Label Codes</h2>
    {{ df_to_html(lc_to_add_to_mb) }}

    <h2 id="wddup">MusicBrainz entities sharing a Wikidata link</h2>
    {{ df_to_html(duplicate_wd) }}

    <h2 id="mbdup">Wikidata entities sharing a MusicBrainz link</h2>
    {{ df_to_html(duplicate_mb) }}

    <h2 id="wd2mb">Add missing Wikidata links to MusicBrainz</h2>
    {{ df_to_html(links_to_add_to_mb) }}

    <h2 id="mb2wd">Add missing MusicBrainz links to Wikidata</h2>
    {{ df_to_html(links_to_add_to_wd) }}
  </body>
</html>
""")

with open('../docs/wd-recordlabels-report.html', 'w') as f:
    f.write(template.render(**globals())
            .replace('&lt;', '<').replace('&gt;', '>')
            .replace('class="dataframe"', 'class="table table-striped table-hover table-sm"')
            .replace('thead', 'thead class="thead-light"'))